## TFX Pipelines

#### Generate Examples

In [ ]:
# Temporary commands to unzip the zip file in Google Cloud Storage
# ! gsutil -m cp gs://text-analysis-323506/train_data/train_val.zip ./
# ! unzip train_val.zip
# ! gunzip *.csv.gz
# ! gsutil -m mv *.csv gs://text-analysis-323506/train_data/

In [ ]:
# ! pip3 install tfx

In [79]:
import time
import os

import tfx

import absl
import os
import tempfile
import time

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tensorflow as tf
from tfx.components.common_nodes.importer_node import ImporterNode

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tfx.components import StatisticsGen, CsvExampleGen, SchemaGen, ExampleValidator, Transform

In [30]:
tf.__version__

'2.6.2'

In [31]:
tfx.__version__

'1.4.0'

In [32]:
DATA_ROOT = 'gs://text-analysis-323506/train_data/'
ARTIFACT_STORE = os.path.join(os.sep, 'home', 'jupyter', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'home', 'jupyter', 'serving_model')

In [33]:
PIPELINE_NAME = 'sentiment-analysis'
PIPELINE_ROOT = os.path.join('/home/jupyter/', PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [34]:
context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

### CSV Example Generator

In [35]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[        
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

In [36]:
example_gen = CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config)

In [ ]:
context.run(example_gen)

In [38]:
examples_uri = example_gen.outputs['examples'].get()[0].uri

In [39]:
tfrecord_filenames = [os.path.join(examples_uri, 'Split-train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'Split-train'))]

In [40]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [41]:
for tfrecord in dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    for name, feature in example.features.feature.items():
        if feature.HasField('bytes_list'):
            value = feature.bytes_list.value
        if feature.HasField('float_list'):
            value = feature.float_list.value
        if feature.HasField('int64_list'):
            value = feature.int64_list.value
        print('{}: {}'.format(name, value))
    print('******')

input: [b'myattorney home business looking account prose litigant state federal lawsuite maintain hisher files']
labels: [1]
******
labels: [0]
input: [b'great book liked book sarah plain tall authors name patricia maclachen characters names caleb sarah anna papa though book interestingit first started caleb siting fire asking questions mama singing songs anna explaining born mama died sarah answered papas letter came live sarah taught caleb swim anybody gets book really enjoy']
******


#### Train and eval datasets have been created properly !

### Statistics Generator

In [42]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

In [ ]:
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### Schema Generator

In [45]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

In [ ]:
context.run(schema_gen)

In [47]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'input',BYTES,required,single,-
'labels',INT,required,single,-


In [48]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

In [49]:
tfdv.set_domain(schema, 'labels', schema_pb2.IntDomain(name='labels', min=0, max=1, is_categorical=True))

In [50]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'input',BYTES,required,single,-
'labels',INT,required,single,min: 0; max: 1


#### Write schema to new file

In [ ]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

### Schema Importer

In [52]:
schema_importer = tfx.dsl.components.common.importer.Importer(
      source_uri=schema_dir,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer') 

In [ ]:
context.run(schema_importer)

In [54]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'input',BYTES,required,single,-
'labels',INT,required,single,min: 0; max: 1


### ExampleValidator

In [58]:
example_validator = ExampleValidator(    
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result']).with_id(
          'example_validator') 

In [ ]:
context.run(example_validator)

In [ ]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "Split-train/SchemaDiff.pb")
!cat $train_anomalies_filename

In [76]:
context.show(example_validator.outputs['anomalies'])

In [80]:
TRANSFORM_MODULE = 'preprocessing.py'
!cat {TRANSFORM_MODULE}

import tensorflow as tf
import tensorflow_transform as tft

import features

def preprocessing_fn(inputs):
    """Preprocesses Covertype Dataset."""

    outputs = {}
    
    print(inputs)
    return outputs

In [92]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=TRANSFORM_MODULE)

In [93]:
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying preprocessing.py -> build/lib
installing to /tmp/tmpwbrrbrig
running install
running install_lib
copying build/lib/preprocessing.py -> /tmp/tmpwbrrbrig
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpwbrrbrig/tfx_user_code_Transform-0.0+4a8cfcaaaf1c647e2ca83293c1b8ddbee7ba552964caeb0c8b091a5ce13bc146-py3.7.egg-info
running install_scripts
creating /tmp/tmpwbrrbrig/tfx_user_code_Transform-0.0+4a8cfcaaaf1

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing /home/jupyter/sentiment-analysis/20211124_163210/_wheels/tfx_user_code_Transform-0.0+4a8cfcaaaf1c647e2ca83293c1b8ddbee7ba552964caeb0c8b091a5ce13bc146-py3-none-any.whl
Processing /home/jupyter/sentiment-analysis/20211124_163210/_wheels/tfx_user_code_Transform-0.0+4a8cfcaaaf1c647e2ca83293c1b8ddbee7ba552964caeb0c8b091a5ce13bc146-py3-none-any.whl
Processing /home/jupyter/sentiment-analysis/20211124_163210/_wheels/tfx_user_code_Transform-0.0+4a8cfcaaaf1c647e2ca83293c1b8ddbee7ba552964caeb0c8b091a5ce13bc146-py3-none-any.whl
called
called
called


called
called
called
called


called
INFO:tensorflow:Assets written to: /home/jupyter/sentiment-analysis/20211124_163210/Transform/transform_graph/11/.temp_path/tftransform_tmp/5517c875ca0b4b97afb2df367ff24b95/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 11
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 30
        type_id: 23
        uri: "/home/jupyter/sentiment-analysis/20211124_163210/Transform/transform_graph/11"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 31
        type_id: 14
        uri: "/home/jupyter/sentiment-analysis/20211124_163210/Transform/transformed_examples/11"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 32
        type_id: 24
        uri: "/home/jupyter/sentiment-analysis/20211124_163210/Transform/updated_analyzer_cache/11"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 33
        type_id: 18
        uri: "/home/jupyter/sentiment-analysis/20211124_163210/Transform/pre_transform_schema/11"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
 

In [ ]:
transform.outputs

In [91]:
transform.outputs['transformed_examples'].get()[0].uri

'/home/jupyter/sentiment-analysis/20211124_163210/Transform/transformed_examples/10'